In [21]:
import pandas as pd
import numpy as np
from data_mani.utils import merge_market_and_gtrends
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor

## Get selected features 

In [2]:
ticker_name =  "AMZN US Equity"
method = "sfi"
out_folder = "nasdaq"
score_path = "results/{}/{}/{}.csv".format(method,out_folder,ticker_name)
scores =  pd.read_csv(score_path)
cut = scores.feature_score.mean()
# cut = scores.feature_score.quantile(0.95)
scores = (scores.loc[scores.feature_score > cut]).feature.to_list()

## Get merged dataframe 

In [12]:
ticker_path = "data/crsp/{}/{}.csv".format(out_folder,ticker_name)
train, test = merge_market_and_gtrends(ticker_path, test_size=0.5)

In [13]:
train.shape, test.shape

((2087, 183), (2087, 183))

In [14]:
words = train.drop("target_return",1).columns.to_list()
max_lag = 20

def add_shift(merged_df, words, max_lag=20):
    for word in tqdm(words, desc="add shift"):
        for shift in range(1, max_lag + 1):
            new_feature = word.replace(" ", "_") + "_{}".format(shift)
            merged_df.loc[:, new_feature] = merged_df[word].shift(shift)

add_shift(merged_df=train, words=words, max_lag=20)
train = train[["target_return"] + scores]
train = train.fillna(0)

add_shift(merged_df=test, words=words, max_lag=20)
test = test[["target_return"] + scores]
test = test.fillna(0)

add shift: 100%|██████████| 182/182 [01:08<00:00,  2.65it/s]


In [15]:
train.shape, test.shape

((2087, 2006), (2087, 2006))

In [26]:
def new_r2(y_true, y_pred):
    erros = y_true - y_pred
    num = (erros).dot(erros)
    dem = (y_true).dot(y_true)
    r2 = 1 - (num / dem)
    return r2

In [31]:
X_train = train.drop("target_return",1).values
y_train = train.target_return.values

X_test = test.drop("target_return",1).values
y_test = test.target_return.values

rf = RandomForestRegressor(n_estimators=100, n_jobs=2)
rf = rf.fit(X_train, y_train)

pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

# pred_train = np.repeat(0, X_train.shape[0])
# pred_test = np.repeat(0, X_test.shape[0])


r2_train = new_r2(y_train, pred_train)
r2_test = new_r2(y_test, pred_test)
print("train", "test")
print(np.round(r2_train, 4), np.round(r2_test, 4))

train test
0.8477 -0.0658
